In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
import time


In [6]:
file_path1 = "../data_enrichment/my_addresses_geocoded.csv"

trip_df = pd.read_csv(file_path1)

# trip_df =  pd.read_csv("../output/trip_advisor_DEFINITIVE.csv")


PER METTERE SOLO I NODI IN NEO4J. 
POI LE RELAZIONI LE CREO CON DELLE QUESRY CIPHER DIRETTAMENTE SU NEO4J.

In [7]:
from neo4j import GraphDatabase

# Connect to Neo4j database
driver = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'progetto'))
session = driver.session()

# Iterate over the rows of the DataFrame
for index, row in trip_df.iterrows():
    # Extract the data from the row
    name_trip = row['name_trip']
    rating_trip = row['rating_trip']
    total_reviews_trip = row['total_reviews_trip']
    cook_type_trip = row['cook_type_trip']
    price_level_trip = row['price_level_trip']
    address_trip = row['address_trip']
    latitude = row['latitude']
    longitude = row['longitude']

    # Create a Neo4j node for the restaurant
    session.run(
        """
        CREATE (restaurant:Restaurant {name: $name, rating: $rating, total_reviews: $total_reviews,
        cook_type: $cook_type, price_level: $price_level, address: $address, latitude: $latitude, longitude: $longitude})
        """,
        name=name_trip,
        rating=rating_trip,
        total_reviews=total_reviews_trip,
        cook_type=cook_type_trip,
        price_level=price_level_trip,
        address=address_trip,
        latitude=latitude,
        longitude=longitude
    )

session.close()
driver.close()


mongodb+srv://mmondini11:<password>@cluster0.xzuticb.mongodb.net/

***per cancellare LE RELAZIONI ma lasciare i nodi caricati*** :
MATCH (:Restaurant)-[r:RECOMMENDS]-(:Restaurant) DELETE r
 
***PER CANCELLARE A BATCH***
optional match (n)-[p:SHARES_CUISINE]->() with n,p LIMIT 300000  delete p

    RACCOMANDAZIONI USANDO TUTTI I PARAMETRI A DISPOSIZIONE.

    *NON MODIFICARE LA SINTASSI, MODIFICARE SOLO I PARAMETRI A PAICERE*


The [0..2] range SERVE PER CONSIDERARE I PRIMI TRE
SE VOGLIAMO UN NUMERO SPECIFICO AGGIUNGIAMO ----------LIMIT 3-------------
TO INTEGER VA PRIMA DI OGNI NUMERO


DA FARE: CERCARE DEI PARAMETRI MIGLIORI E NON A CASO

**QUI UNA CUCINE BASTA CHE MATCH CON ALMENO UNA CUCINA DELLA RACCOMANDAZIONE  E DISTANZA MAX 3KM, in più c'è la clausoa fast food**




MATCH (r1:Restaurant), (r2:Restaurant)
WHERE r1 <> r2 
  AND ((r1.cook_type = 'Fast Food' AND r2.cook_type = 'Fast Food') OR
       (r1.cook_type <> 'Fast Food' AND r2.cook_type <> 'Fast Food' AND
        (ANY(cuisine IN split(r1.cook_type, ',') WHERE r2.cook_type CONTAINS cuisine) OR
         ANY(cuisine IN split(r2.cook_type, ',') WHERE r1.cook_type CONTAINS cuisine))))
  AND toInteger(r1.price_level) >= toInteger(r2.price_level) - 1 AND toInteger(r1.price_level) <= toInteger(r2.price_level) + 1
  AND toFloat(r1.rating) >= toFloat(r2.rating) - 0.5 AND toFloat(r1.rating) <= toFloat(r2.rating) + 0.5
  AND toInteger(r1.total_reviews) >= toInteger(r2.total_reviews) - 50 AND toInteger(r1.total_reviews) <= toInteger(r2.total_reviews) + 50
  AND toFloat(r1.latitude) IS NOT NULL AND toFloat(r1.longitude) IS NOT NULL
  AND toFloat(r2.latitude) IS NOT NULL AND toFloat(r2.longitude) IS NOT NULL
  AND toFloat(r1.latitude) >= -90 AND toFloat(r1.latitude) <= 90
  AND toFloat(r1.longitude) >= -180 AND toFloat(r1.longitude) <= 180
  AND toFloat(r2.latitude) >= -90 AND toFloat(r2.latitude) <= 90
  AND toFloat(r2.longitude) >= -180 AND toFloat(r2.longitude) <= 180
  AND distance(point({ latitude: toFloat(r1.latitude), longitude: toFloat(r1.longitude) }), point({ latitude: toFloat(r2.latitude), longitude: toFloat(r2.longitude) })) <= 3000
WITH r1, r2, rand() AS random
ORDER BY random
WITH r1, COLLECT(r2) AS recommendedRestaurants
FOREACH (recRestaurant IN recommendedRestaurants[0..2] |
  MERGE (r1)-[:RECOMMENDS]->(recRestaurant)
)
RETURN r1, COLLECT(recommendedRestaurants[0..2]) AS recommendations


In [ ]:
# query9= '''
# MATCH (r1:Restaurant), (r2:Restaurant)
# WHERE r1 <> r2 
#   AND ((r1.cook_type = 'Fast Food' AND r2.cook_type = 'Fast Food') OR
#        (r1.cook_type <> 'Fast Food' AND r2.cook_type <> 'Fast Food' AND
#         (ANY(cuisine IN split(r1.cook_type, ',') WHERE r2.cook_type CONTAINS cuisine) OR
#          ANY(cuisine IN split(r2.cook_type, ',') WHERE r1.cook_type CONTAINS cuisine))))
#   AND toInteger(r1.price_level) >= toInteger(r2.price_level) - 1 AND toInteger(r1.price_level) <= toInteger(r2.price_level) + 1
#   AND toFloat(r1.rating) >= toFloat(r2.rating) - 0.5 AND toFloat(r1.rating) <= toFloat(r2.rating) + 0.5
#   AND toInteger(r1.total_reviews) >= toInteger(r2.total_reviews) - 50 AND toInteger(r1.total_reviews) <= toInteger(r2.total_reviews) + 50
#   AND toFloat(r1.latitude) IS NOT NULL AND toFloat(r1.longitude) IS NOT NULL
#   AND toFloat(r2.latitude) IS NOT NULL AND toFloat(r2.longitude) IS NOT NULL
#   AND toFloat(r1.latitude) >= -90 AND toFloat(r1.latitude) <= 90
#   AND toFloat(r1.longitude) >= -180 AND toFloat(r1.longitude) <= 180
#   AND toFloat(r2.latitude) >= -90 AND toFloat(r2.latitude) <= 90
#   AND toFloat(r2.longitude) >= -180 AND toFloat(r2.longitude) <= 180
#   AND point.distance(point({ latitude: toFloat(r1.latitude), longitude: toFloat(r1.longitude) }), point({ latitude: toFloat(r2.latitude), longitude: toFloat(r2.longitude) })) <= 3000
# WITH r1, r2, rand() AS random
# ORDER BY random
# WITH r1, COLLECT(r2) AS recommendedRestaurants
# FOREACH (recRestaurant IN recommendedRestaurants[0..2] |
#   MERGE (r1)-[:RECOMMENDS]->(recRestaurant)
# )
# RETURN r1, COLLECT(recommendedRestaurants[0..2]) AS recommendations

# '''




# from py2neo import Graph
# from py2neo.data import Node, Relationship
# from pprint import pprint


# graph = Graph("bolt://localhost:7687", auth=("neo4j", "progetto"))
# graph.run(query9)
# cursor = graph.run(query9)

In [ ]:
# from py2neo import Graph

# # Set up the graph connection
# graph = Graph("bolt://localhost:7687", auth=("neo4j", "maurino"))

# # Define the pagination parameters
# page_size = 100  # Number of results per page
# page_number = 1  # Page number to retrieve (starting from 1)

# # Calculate the skip value based on the page size and number
# skip = (page_number - 1) * page_size

# # Construct the modified query with pagination
# query9 = '''
# MATCH (r1:Restaurant), (r2:Restaurant)
# WHERE r1 <> r2 
#   AND ((r1.cook_type = 'Fast Food' AND r2.cook_type = 'Fast Food') OR
#        (r1.cook_type <> 'Fast Food' AND r2.cook_type <> 'Fast Food' AND
#         (ANY(cuisine IN split(r1.cook_type, ',') WHERE r2.cook_type CONTAINS cuisine) OR
#          ANY(cuisine IN split(r2.cook_type, ',') WHERE r1.cook_type CONTAINS cuisine))))
#   AND toInteger(r1.price_level) >= toInteger(r2.price_level) - 1 AND toInteger(r1.price_level) <= toInteger(r2.price_level) + 1
#   AND toFloat(r1.rating) >= toFloat(r2.rating) - 0.5 AND toFloat(r1.rating) <= toFloat(r2.rating) + 0.5
#   AND toInteger(r1.total_reviews) >= toInteger(r2.total_reviews) - 50 AND toInteger(r1.total_reviews) <= toInteger(r2.total_reviews) + 50
#   AND toFloat(r1.latitude) IS NOT NULL AND toFloat(r1.longitude) IS NOT NULL
#   AND toFloat(r2.latitude) IS NOT NULL AND toFloat(r2.longitude) IS NOT NULL
#   AND toFloat(r1.latitude) >= -90 AND toFloat(r1.latitude) <= 90
#   AND toFloat(r1.longitude) >= -180 AND toFloat(r1.longitude) <= 180
#   AND toFloat(r2.latitude) >= -90 AND toFloat(r2.latitude) <= 90
#   AND toFloat(r2.longitude) >= -180 AND toFloat(r2.longitude) <= 180
#   AND point.distance(point({ latitude: toFloat(r1.latitude), longitude: toFloat(r1.longitude) }), point({ latitude: toFloat(r2.latitude), longitude: toFloat(r2.longitude) })) <= 3000
# WITH r1, r2, rand() AS random
# ORDER BY random
# WITH r1, COLLECT(r2) AS recommendedRestaurants
# SKIP $skip
# LIMIT $limit
# RETURN r1, COLLECT(recommendedRestaurants[0..2]) AS recommendations
# '''

# # Execute the modified query with pagination
# page_results = graph.run(query9, skip=skip, limit=page_size).data()

# # Print the results of the current page
# for record in page_results:
#     print(record)


In [ ]:

#MATHCARE CON LO SCORE

from py2neo import Graph
from py2neo.data import Node, Relationship
from pprint import pprint

query7 = '''
MATCH (r1:Restaurant), (r2:Restaurant)
WHERE r1 <> r2 
  AND ((r1.cook_type = 'Fast Food' AND r2.cook_type = 'Fast Food') OR
       (r1.cook_type <> 'Fast Food' AND r2.cook_type <> 'Fast Food' AND
        (ANY(cuisine IN split(r1.cook_type, ',') WHERE r2.cook_type CONTAINS cuisine) OR
         ANY(cuisine IN split(r2.cook_type, ',') WHERE r1.cook_type CONTAINS cuisine))))
  AND toInteger(r1.price_level) >= toInteger(r2.price_level) - 0.5 AND toInteger(r1.price_level) <= toInteger(r2.price_level) + 0.5
  AND toFloat(r1.rating) >= toFloat(r2.rating) - 0.5 AND toFloat(r1.rating) <= toFloat(r2.rating) + 0.5
  AND toInteger(r1.total_reviews) >= toInteger(r2.total_reviews) - 30 AND toInteger(r1.total_reviews) <= toInteger(r2.total_reviews) + 30
  AND toFloat(r1.latitude) IS NOT NULL AND toFloat(r1.longitude) IS NOT NULL
  AND toFloat(r2.latitude) IS NOT NULL AND toFloat(r2.longitude) IS NOT NULL
  AND toFloat(r1.latitude) >= -90 AND toFloat(r1.latitude) <= 90
  AND toFloat(r1.longitude) >= -180 AND toFloat(r1.longitude) <= 180
  AND toFloat(r2.latitude) >= -90 AND toFloat(r2.latitude) <= 90
  AND toFloat(r2.longitude) >= -180 AND toFloat(r2.longitude) <= 180
  AND point.distance(point({ latitude: toFloat(r1.latitude), longitude: toFloat(r1.longitude) }), point({ latitude: toFloat(r2.latitude), longitude: toFloat(r2.longitude) })) <= 3000
WITH r1, r2, rand() AS random,
     abs(toInteger(r1.price_level) - toInteger(r2.price_level)) AS price_diff,
     abs(toFloat(r1.rating) - toFloat(r2.rating)) AS rating_diff,
     abs(toInteger(r1.total_reviews) - toInteger(r2.total_reviews)) AS review_diff
ORDER BY random
WITH r1, COLLECT(r2) AS recommendedRestaurants, 
     MIN(price_diff) AS min_price_diff, 
     MIN(rating_diff) AS min_rating_diff, 
     MIN(review_diff) AS min_review_diff
WITH r1, recommendedRestaurants, 
     CASE WHEN min_price_diff = 0 THEN 1 ELSE (abs(toInteger(r1.price_level) - toInteger(r1.price_level)) / min_price_diff) END AS price_score, 
     CASE WHEN min_rating_diff = 0 THEN 1 ELSE (abs(toFloat(r1.rating) - toFloat(r1.rating)) / min_rating_diff) END AS rating_score, 
     CASE WHEN min_review_diff = 0 THEN 1 ELSE (abs(toInteger(r1.total_reviews) - toInteger(r1.total_reviews)) / min_review_diff) END AS review_score
WITH r1, recommendedRestaurants, 
     (price_score + rating_score + review_score) AS total_score
ORDER BY total_score DESC
WITH r1, COLLECT(recommendedRestaurants[0..2]) AS recommendations
FOREACH (recRestaurant IN recommendations |
  FOREACH (rec IN recRestaurant |
    MERGE (r1)-[:RECOMMENDS]->(rec)
  )
)
RETURN r1, recommendations
'''
graph = Graph("bolt://localhost:7687", auth=("neo4j", "progetto"))
graph.run(query7)
cursor = graph.run(query7)
# print(cursor.to_table())
